* changed by nov05 on 2024-11-28  
* Udacity AWS MLE Nanodegree (ND189)  
  Course 4, 3.7 Exercise: Training a Convolutional Neural Network  
* `conda activate drlnd_py310` with cuda enabled   

In [1]:
%cd ..
%pwd

d:\github\udacity-CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter


d:\Users\guido\miniconda3\envs\drlnd_py310\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'd:\\github\\udacity-CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter'

## Solution: Training a CNN

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.optim.lr_scheduler import CosineAnnealingLR
import time
from tqdm import tqdm

In [8]:
def test(model, test_loader, criterion, device):
    print("Testing model on whole testing dataset...")
    model.eval()
    running_loss=0
    running_corrects=0
    
    for inputs, labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds==labels.data).item()

    total_loss = running_loss / len(test_loader.dataset)
    total_acc = running_corrects/ len(test_loader.dataset)
    print(f"🟢 Testing Accuracy: {100*total_acc}, Testing Loss: {total_loss}")
    

def train(model, train_loader, validation_loader, criterion, optimizer, device,
          epochs):

    best_loss=1e6
    image_dataset={'train':train_loader, 'eval':validation_loader}
    loss_counter=0
    
    for epoch in tqdm(range(epochs)):
        for phase in ['train', 'eval']:
            print(f"Epoch {epoch}, Phase {phase}")
            if phase=='train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            running_samples=0

            for step, (inputs, labels) in enumerate(image_dataset[phase]):
                inputs=inputs.to(device)
                labels=labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase=='train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==labels.data).item()
                running_samples += len(inputs)
                if (running_samples%2000)==0:
                    accuracy = running_corrects/running_samples
                    print("Step {}, Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%) Time: {}".format(
                            step,
                            running_samples,
                            len(image_dataset[phase].dataset),
                            100.0 * (running_samples / len(image_dataset[phase].dataset)),
                            loss.item(),
                            running_corrects,
                            running_samples,
                            100.0*accuracy,
                            time.asctime() # for measuring time for testing, remove for students and in the formatting
                        )
                    )
                ## NOTE: Comment lines below to train and test on whole dataset
                # if running_samples>(0.2*len(image_dataset[phase].dataset)):
                #     break

            epoch_loss = running_loss / running_samples
            # epoch_acc = running_corrects / running_samples
            
            if phase=='eval':
                if epoch_loss<best_loss:
                    best_loss=epoch_loss
                else:
                    loss_counter+=1
        ## early stopping
        if loss_counter==2:
            break
    return model

def create_model():
    # model = resnet18(pretrained=True)
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    for param in model.parameters():
        param.requires_grad = False   
    num_features=model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_features, 10)
    )
    return model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"👉 Running on Device {device}")

training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
testing_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


epochs=100
batch_size=128
model=create_model()
model=model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.fc.parameters(), lr=0.001, weight_decay=5e-4)
# optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

trainset = CIFAR10(root='./data', train=True,
    download=True, transform=training_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
    shuffle=True)
testset = CIFAR10(root='./data', train=False,
    download=True, transform=testing_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
    shuffle=False)

train(model, trainloader, testloader, criterion, optimizer, device, epochs)
## Epoch 0, Phase train
## Images [2000/50000 (4%)] Loss: 1.16 Accuracy: 846/2000 (42.30%) Time: Thu Nov 28 16:48:06 2024

👉 Running on Device cuda:0
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0, Phase train
Images [16000/50000 (32%)] Loss: 0.81 Accuracy: 9755/16000 (60.97%) Time: Thu Nov 28 17:09:15 2024
Images [32000/50000 (64%)] Loss: 0.87 Accuracy: 21742/32000 (67.94%) Time: Thu Nov 28 17:09:59 2024
Images [48000/50000 (96%)] Loss: 0.60 Accuracy: 34069/48000 (70.98%) Time: Thu Nov 28 17:11:18 2024
Images [50000/50000 (100%)] Loss: 0.61 Accuracy: 35639/50000 (71.28%) Time: Thu Nov 28 17:11:31 2024
Epoch 0, Phase eval


  1%|          | 1/100 [03:51<6:22:32, 231.84s/it]

Images [10000/10000 (100%)] Loss: 0.69 Accuracy: 7846/10000 (78.46%) Time: Thu Nov 28 17:12:23 2024
Epoch 1, Phase train
Images [16000/50000 (32%)] Loss: 0.65 Accuracy: 12512/16000 (78.20%) Time: Thu Nov 28 17:13:32 2024
Images [32000/50000 (64%)] Loss: 0.76 Accuracy: 25172/32000 (78.66%) Time: Thu Nov 28 17:14:44 2024
Images [48000/50000 (96%)] Loss: 0.49 Accuracy: 37889/48000 (78.94%) Time: Thu Nov 28 17:15:55 2024
Images [50000/50000 (100%)] Loss: 0.58 Accuracy: 39499/50000 (79.00%) Time: Thu Nov 28 17:16:05 2024
Epoch 1, Phase eval


...

  7%|▋         | 7/100 [31:14<6:53:24, 266.71s/it]

Images [10000/10000 (100%)] Loss: 0.52 Accuracy: 8024/10000 (80.24%) Time: Thu Nov 28 17:39:46 2024
Epoch 7, Phase train
Images [16000/50000 (32%)] Loss: 0.55 Accuracy: 12997/16000 (81.23%) Time: Thu Nov 28 17:40:50 2024
Images [32000/50000 (64%)] Loss: 0.53 Accuracy: 26036/32000 (81.36%) Time: Thu Nov 28 17:41:54 2024
Images [48000/50000 (96%)] Loss: 0.44 Accuracy: 38964/48000 (81.17%) Time: Thu Nov 28 17:43:00 2024
Images [50000/50000 (100%)] Loss: 0.70 Accuracy: 40586/50000 (81.17%) Time: Thu Nov 28 17:43:07 2024
Epoch 7, Phase eval


  7%|▋         | 7/100 [35:15<7:48:22, 302.18s/it]

Images [10000/10000 (100%)] Loss: 0.48 Accuracy: 8086/10000 (80.86%) Time: Thu Nov 28 17:43:47 2024


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
test(model, testloader, criterion, device)
## Testing Accuracy: 80.86, Testing Loss: 0.5680697995185852
## 24.5s

Testing model on whole testing dataset...
🟢 Testing Accuracy: 80.86, Testing Loss: 0.5680697995185852
